In [ ]:
pip install -q transformers accelerate peft bitsandbytes datasets trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 13.2 MB/s eta 0:00:00


In [ ]:
  import torch
  from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    trust_remote_code=True
)

tokenizer.pad_token = tokenizer.eos_token



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

model.config.use_cache = False


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

### peft LORA

In [ ]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

In [ ]:
model = prepare_model_for_kbit_training(model)


In [ ]:
lora_config = LoraConfig(
    r=16,                       # rank (good balance)
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


In [ ]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 41,943,040 || all params: 7,283,675,136 || trainable%: 0.5758


In [ ]:
import torch
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False

/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)


In [ ]:
from datasets import Dataset
import json

def format_example(example):
    prompt = f"""### Instruction:
Generate a multiple-choice question.

Subject: {example['subject']}
Topic: {example['topic']}
Difficulty: {example['difficulty']}

### Response:
"""

    options = "\n".join(
        [f"{chr(65+i)}. {opt}" for i, opt in enumerate(example["options"])]
    )

    answer = chr(65 + example["correct_option_index"])

    response = f"""Question: {example['question']}

Options:
{options}

Correct Answer: {answer}

Explanation:
{example['explanation']}
"""

    return {"text": prompt + response}

# Load JSON
with open("/content/ML_DL_mcqs.json") as f:
    raw_data = json.load(f)

dataset = Dataset.from_list(raw_data)
dataset = dataset.map(format_example, remove_columns=dataset.column_names)


Map:   0%|          | 0/2579 [00:00<?, ? examples/s]

In [ ]:
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        max_length=1024,
        padding=False,
    )

tokenized_dataset = dataset.map(
    tokenize,
    batched=True,
    remove_columns=["text"]
)


Map:   0%|          | 0/2579 [00:00<?, ? examples/s]

In [ ]:
model.gradient_checkpointing_enable()
model.enable_input_require_grads()


In [ ]:
print(dataset.column_names)


['text']


In [ ]:
print("BF16 supported:", torch.cuda.is_bf16_supported())
print("GPU:", torch.cuda.get_device_name())


BF16 supported: True
GPU: Tesla T4


In [ ]:
def formatting_func(example):
    return example["text"]


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./quiz-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=20,
    save_steps=500,
    save_total_limit=2,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    formatting_func=formatting_func,  # 👈 ONLY special arg allowed
)


Applying formatting function to train dataset:   0%|          | 0/2579 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/2579 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2579 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2579 [00:00<?, ? examples/s]

In [ ]:
trainer.train()


Step,Training Loss
20,1.006700
40,0.553700
60,0.491800
80,0.495200
100,0.477900
120,0.471500
140,0.455400
160,0.474000
180,0.437600
200,0.452000


TrainOutput(global_step=969, training_loss=0.28927160072129826, metrics={'train_runtime': 11332.4349, 'train_samples_per_second': 0.683, 'train_steps_per_second': 0.086, 'total_flos': 4.789923796662682e+16, 'train_loss': 0.28927160072129826, 'entropy': 0.16335533414758854, 'num_tokens': 1116126.0, 'mean_token_accuracy': 0.9598172671759306, 'epoch': 3.0})

In [ ]:
trainer.model.save_pretrained("quiz-lora-adapter")
tokenizer.save_pretrained("quiz-lora-adapter")

('quiz-lora-adapter/tokenizer_config.json',
 'quiz-lora-adapter/special_tokens_map.json',
 'quiz-lora-adapter/tokenizer.model',
 'quiz-lora-adapter/added_tokens.json',
 'quiz-lora-adapter/tokenizer.json')

In [ ]:
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    quantization_config=bnb_config,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, "quiz-lora-adapter")
model.eval()

prompt = """### Instruction:
Generate a multiple-choice question.

Subject: Deep Learning
Topic: Convolutional Neural Networks
Difficulty: Medium

### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )

print(tokenizer.decode(output[0], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### Instruction:
Generate a multiple-choice question.

Subject: Deep Learning
Topic: Convolutional Neural Networks
Difficulty: Medium

### Response:
Question: What is the primary purpose of the Softmax activation function?

Options:
A. To introduce non-linearity
B. To reduce spatial dimensions
C. To compute class probabilities
D. To apply padding

Correct Answer: C

Explanation:
The Softmax activation function is used to compute class probabilities, assigning a probability to each class in a multi-class classification problem.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
